Establishing a connection to the MySQL database

In [15]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    user='root',
    password='Sursurpatil@4',
    host='127.0.0.1',
    database='user_analytics'
)

cursor = conn.cursor()


In [16]:
DATA_PATH = r'E:\Project5\telcom_data p5.csv'
data = pd.read_csv(DATA_PATH)

Loading the saved data from Task 4

In [17]:
import pickle

with open('task4_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [18]:
for index, row in data.head(10).iterrows():
 print(f"Engagement: {row['engagement_score']}, Experience: {row['experience_score']}, Satisfaction: {row['satisfaction_score']}")

Engagement: 7528.192032213845, Experience: 1.0, Satisfaction: 843408406.6170448
Engagement: 865.5280196435468, Experience: 1.0, Satisfaction: 126037214.509565
Engagement: 4415.580488852996, Experience: 1.0, Satisfaction: 558045719.8272809
Engagement: 8467.073619631901, Experience: 1.0, Satisfaction: 402506796.99558675
Engagement: 39279.07891332471, Experience: 1.0, Satisfaction: 1366386971.0257843
Engagement: 2422.2771642196526, Experience: 1.0, Satisfaction: 558441641.8508786
Engagement: 5100.678295419134, Experience: 1.0, Satisfaction: 562311119.5865241
Engagement: 3850.280176853899, Experience: 1.0, Satisfaction: 296604270.19490236
Engagement: 1997.4343234443274, Experience: 1.0, Satisfaction: 901430037.6894312
Engagement: 5866.562232687779, Experience: 1.0, Satisfaction: 686988638.9362719


In [19]:
for index, row in data.iterrows():
    sql = "INSERT INTO user_data (engagement_score, experience_score, satisfaction_score) VALUES (%s, %s, %s)"
    val = (float(row['engagement_score']), float(row['experience_score']), float(row['satisfaction_score']))
    cursor.execute(sql, val)

conn.commit()
cursor.close()
conn.close()

In [20]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Split data into training and testing sets
X = user_behavior[['engagement_score', 'experience_score']]
y = user_behavior['satisfaction_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_components", pca.n_components)
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)

    # Train model
    model = LinearRegression()
    model.fit(X_train_pca, y_train)

    # Make predictions
    y_pred = model.predict(X_test_pca)

    # Log metrics
    mse = mean_squared_error(y_test, y_pred)
    mlflow.log_metric("mse", mse)

    # Log artifacts
    mlflow.sklearn.log_model(model, "model")



NameError: name 'user_behavior' is not defined